#### Importation des bibliothèques nécessaires

In [1]:
from neo4j import GraphDatabase
import folium

#### initialisation de la connexion à la base de données Neo4j

In [2]:
uri = "neo4j://localhost:7687"
username = "neo4j"
password = "bdsas1234"
driver = GraphDatabase.driver(uri, auth=(username, password))
session= driver.session()

#### Exécution d'une requête Cypher pour récupérer tous les nœuds de type "Node" dans la base de données Neo4j

In [3]:
resultats = session.run("MATCH (n:Node) RETURN n")
i=0
for record in resultats:
    print(record,'\n')
    i+=1
print(i)

<Record n=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6249' labels=frozenset({'Node'}) properties={'lon': -4.9969476, 'id': 6819872768, 'lat': 34.0305657}>> 

<Record n=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6250' labels=frozenset({'Node'}) properties={'lon': -5.0240188, 'id': 1584652349, 'lat': 34.020084}>> 

<Record n=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6251' labels=frozenset({'Node'}) properties={'lon': -4.9706572, 'id': 5743935549, 'lat': 34.0145836}>> 

<Record n=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6252' labels=frozenset({'Node'}) properties={'lon': -5.0242265, 'id': 1584652351, 'lat': 34.02013}>> 

<Record n=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6253' labels=frozenset({'Node'}) properties={'lon': -5.0194953, 'id': 1588101188, 'lat': 34.0176181}>> 

<Record n=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6254' labels=frozenset({'Node'}) properties={'lon': -4.9709486, 'id': 574393

In [4]:
resultats = session.run("""
    MATCH (n:Node {id: $node_id})-[r:CONNECTED_TO]->(successeur)
    RETURN successeur.id AS id_successeur
""", node_id=1587249584)
successeurs = [record["id_successeur"] for record in resultats]
print("Successeurs du nœud", successeurs)

Successeurs du nœud [1587249587, 1587249580, 1587249581]


#### Récupère l'ID d'un nœud dans la base de données Neo4j en fonction de sa latitude et sa longitude.

In [5]:
def getNode(lat,lon):
    result = session.run("MATCH (n:Node) WHERE n.lat = $lat AND n.lon = $lon RETURN n.id AS id", lat=lat, lon=lon)
    record = result.single()
    return record["id"]

In [6]:
resultHotels = session.run("MATCH (h) WHERE h.etat = $etat RETURN h.id AS id", etat="hotel")
i=0
for tmp in resultHotels:
    print(tmp['id'])
#     i+=1
# print(i)


257523827
3189100648
257523827
5754364187
5159327313
257523827
5744533861
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
257523827
6809823682
257523827
257523827
6869127398
257523827
260148739
257523827
260148609
4579373623
257523827
6801921103
5744533829
257523827
5752692143
3971607478
257523827
257523827
1588811310
1852214306
1588811421
260258986
257523827
260146380
1588811310
260141412
1852214233
257523827
1851617467
6822378399
4587733689
257523827
1587596252
5744533872
257523827
6586283008
257523827
257523827
257523827
257523827
26220408

In [7]:
print(getNode(34.0305657,-4.9969476))

6819872768


#### Trouve le chemin le plus court entre deux nœuds dans la base de données Neo4j en utilisant l'algorithme de Dijkstra.

In [8]:
def trouverPlusCourtChemin(tx,depart_id, arrivee_id):
    # Exécute une requête Cypher pour trouver le chemin le plus court entre les nœuds de départ et d'arrivée
    resultats = tx.run("""
            MATCH (depart:Node {id: $depart_id}), (arrivee:Node {id: $arrivee_id})
            CALL apoc.algo.dijkstra(depart, arrivee, 'CONNECTED_TO', 'distance')  YIELD path, weight
            RETURN path AS chemin, weight AS cout
           """, depart_id=depart_id, arrivee_id=arrivee_id)
    return resultats.single()

<p>Nous cherchions à calculer les chemins les plus courts entre un nœud de départ, déterminé par ses coordonnées, et une liste d'hôtels dans notre base de données Neo4j. Pour ce faire, nous avons utilisé la fonction que nous avions précédemment définie, 'trouverPlusCourtChemin'</p>

In [9]:
resultHotels = session.run("MATCH (h) WHERE h.etat = $etat RETURN h.id AS id", etat="hotel")
departNode=(34.0351607,-4.9802339)
departId =getNode(departNode[0],departNode[1])
chemins=[]
i=0
for hot in resultHotels:
    chemin = session.execute_read(trouverPlusCourtChemin, departId, hot['id'])
    chemins.append((chemin,chemin['cout']))
    i+=1
    print(i)
    if i>15:
        break
    # print(departId, hot['id'],chemin['cout'])

/home/abderrahim/anaconda3/lib/python3.11/site-packages/neo4j/_sync/work/result.py:500: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


Nous cherchions à trouver le chemin le plus court parmi une liste de chemins et de coûts stockés dans la variable 'chemins'. Le code que nous avons écrit est correct. Il utilise la fonction 'min' de Python avec une fonction lambda pour déterminer le chemin ayant le coût minimum.

#### Trouver le chemin avec le coût minimum

In [10]:
chemin_min = min(chemins, key=lambda x: x[1])
chemin_min

(<Record chemin=<Path start=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:6367' labels=frozenset({'Node'}) properties={'lon': -4.9802339, 'id': 6925700301, 'lat': 34.0351607}> end=<Node element_id='4:884f68fd-ef14-4826-a0fc-522a75037427:7337' labels=frozenset({'Node'}) properties={'lon': -4.9961565, 'id': 5159327313, 'lat': 34.041511}> size=28> cout=2148.56>,
 2148.56)

#### Extraire les nœuds du chemin minimum

In [11]:
cheminList=[]
for tmp in chemin_min[0]['chemin'].nodes:
    cheminList.append(dict(tmp))
    # print(dict(tmp))

Nous avions l'intention de créer une liste de points à partir d'une liste de nœuds, où chaque point inclut l'ID du nœud ainsi que ses coordonnées (latitude, longitude).

In [12]:
points=[]
for node in cheminList:
    points.append([node['id'],(node['lat'],node['lon'])])
points

[[6925700301, (34.0351607, -4.9802339)],
 [6925700296, (34.0352025, -4.9802762)],
 [260262872, (34.0346069, -4.9820017)],
 [260262871, (34.0345262, -4.9822399)],
 [6799332844, (34.0343376, -4.9827798)],
 [4579374531, (34.0342595, -4.9830035)],
 [6925607079, (34.0342067, -4.9831611)],
 [260262492, (34.0340759, -4.9855587)],
 [6800043502, (34.0341651, -4.9860731)],
 [262836956, (34.0342412, -4.9865903)],
 [6925609408, (34.034732, -4.9870842)],
 [6925609420, (34.0353917, -4.9874175)],
 [6925609438, (34.036119, -4.9875782)],
 [4579373882, (34.036509, -4.9877666)],
 [4579372778, (34.0384844, -4.9888706)],
 [260142564, (34.0386968, -4.9889999)],
 [260142566, (34.0401383, -4.9897939)],
 [6093588041, (34.0399915, -4.9903331)],
 [1751425226, (34.0399059, -4.9906428)],
 [5735102881, (34.0392784, -4.9929503)],
 [4872446719, (34.0391677, -4.9933267)],
 [260142571, (34.0391159, -4.9934704)],
 [260146747, (34.0390154, -4.9936684)],
 [260146746, (34.0394101, -4.9939792)],
 [260146742, (34.0404185, -4

Exécutera la requête Cypher pour récupérer l'hôtel correspondant à l'ID du dernier point dans la liste points

In [13]:
resultHotels = session.run("MATCH (h) WHERE h.id=$id and h.etat = $etat RETURN h", id=points[-1][0],etat="hotel")
for tmp in resultHotels:
    nameHotel=tmp
print(nameHotel['h']['name']+'\n'+nameHotel['h']['address'])

Youth Hostel Fez
18 Rue Abessalem Serghini, Fes 30000, Maroc


### Afficher le chemin sur la carte fes.

Nous avons construit une carte Folium avec des marqueurs pour le nœud de départ, le dernier point de notre chemin, ainsi qu'une ligne reliant tous les points du chemin. De plus, nous avons ajouté un marqueur pour l'hôtel correspondant au dernier point de notre chemin, avec une fenêtre contextuelle affichant des informations sur l'hôtel.

In [14]:
coordonnees = [coords for node_id, coords in points]
map = folium.Map(location=(34.0305657,-4.9969476),zoom_start=13)
folium.Marker(location=departNode,popup="depart node").add_to(map)
textHtml="""
         <div style="width:250px; padding-bottom:10px;">
           <b><span style="color:red;">le nom : </span>"""+nameHotel['h']['name']+"""</b><br>
           <b><span style="color:red;">L'adresse :</span> """+nameHotel['h']['address']+"""</b><br>
           <b><span style="color:red;">le numéro de téléphone :</span> """+nameHotel['h']['phoneNumber']+"""</b><br>
           <b><span style="color:red;">Évaluation :</span> """+nameHotel['h']['rating']+"""</b><br><br><br>
           <img src='"""+nameHotel['h']['imgUrl']+"""' style="width:200px;display: block;margin: 0 auto;">
          </div>"""
folium.Marker(location=coordonnees[-1],popup=folium.Popup(textHtml)).add_to(map)
folium.PolyLine(locations=coordonnees, color='blue').add_to(map)
map

In [17]:
import tkinter as tk
from PIL import Image, ImageTk
import requests
from io import BytesIO

def aficherHotel(nameHotel):
# Initialize the main window
    root = tk.Tk()
    root.title(nameHotel['h']['name'])
    root.geometry("400x250")
    clolorBack="pink"
    root.configure(bg=clolorBack)
    # Create a frame to hold the content
    content_frame = tk.Frame(root, width=250,bg=clolorBack)
    content_frame.pack(pady=10, padx=10)
    
    # Create and pack labels for hotel information
    name_label = tk.Label(content_frame, text=f"le nom: {nameHotel['h']['name']}", fg="white", font=("Helvetica", 12, "bold"),bg=clolorBack)
    name_label.pack(anchor='w')
    
    address_label = tk.Label(content_frame, text=f"L'adresse: {nameHotel['h']['address']}", fg="white", font=("Helvetica", 12, "bold"),bg=clolorBack)
    address_label.pack(anchor='w')
    
    phone_label = tk.Label(content_frame, text=f"le numéro de téléphone: {nameHotel['h']['phoneNumber']}", fg="white", font=("Helvetica", 12, "bold"),bg=clolorBack)
    phone_label.pack(anchor='w')
    
    phone_label = tk.Label(content_frame, text=f"Évaluation: {nameHotel['h']['rating']}", fg="white", font=("Helvetica", 12, "bold"),bg=clolorBack)
    phone_label.pack(anchor='w')
    
    # # # Fetch and display the image
    # # response = requests.get("https://lh5.googleusercontent.com/p/AF1QipOvKi_QMEFyhFQsG2Ix1iuQW7NWbhmvXhfXNqX2=w408-h408-k-no")
    # try:
    #     response = requests.get(nameHotel['h']['imgUrl'])
    #     response.raise_for_status()  # Raise an error for bad status codes
    #     img_data = response.content
    #     img = Image.open(BytesIO(img_data))
    #     img = img.resize((200, 150), Image.LANCZOS)
    #     photo = ImageTk.PhotoImage(img)
        
    #     image_label = tk.Label(content_frame, image=photo)
    #     image_label.image = photo  # Keep a reference to avoid garbage collection
    #     image_label.pack(pady=10)
    # except (requests.RequestException, IOError) as e:
    #     print(f"Error fetching image: {e}")
    #     img = Image.new('RGB', (200, 150), color='gray')  # Placeholder image
    #     photo = ImageTk.PhotoImage(img)
   
    # Run the Tkinter main loop
    root.mainloop()


In [18]:
from tkinter import *
import tkintermapview

def on_marker_click(event):
        aficherHotel(nameHotel)
        print("Marker clicked!")

root = Tk()
root.title('bigdata')
root.geometry("900x600")

my_label = LabelFrame(root)
my_label.pack(pady=20)

map_widget = tkintermapview.TkinterMapView(my_label, width=900, height=600, corner_radius=0)
map_widget.set_position(34.0305657,-4.9969476)

map_widget.set_marker(departNode[0],departNode[1], text="depart", text_color="green",
                                marker_color_circle="blue", marker_color_outside="red", font=("Helvetica Bold", 10))

map_widget.set_marker(coordonnees[-1][0],coordonnees[-1][1], text=nameHotel['h']['name'], text_color="black",
                                marker_color_circle="blue", marker_color_outside="red", font=("Helvetica Bold", 12))

map_widget.canvas.bind("<Button-1>", on_marker_click)

for i in range(len(coordonnees)-1):
    polyline_coords=[coordonnees[i],coordonnees[i+1]]
    map_widget.set_polygon(polyline_coords)

map_widget.set_zoom(13)
map_widget.pack()
root.mainloop()

invalid command name "136805455596800update_canvas_tile_images"
    while executing
"136805455596800update_canvas_tile_images"
    ("after" script)
